In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Annulus, Rectangle
import ipywidgets
%matplotlib widget

In [ ]:
# Spider math
# Can assume struts span ~10m at the top end assembly.
# They're at 45-degree angles roughly
# Use the Zemax vals; they pass through horizontal points
# (0., 0.5656854249)
# which means they are sqrt(2)*0.56... = 0.8 m apart

In [ ]:
# Characterize by:
# (xy) = (+/- 0.4, +/- 2.5) and transpositions
# width = 0.05
# length = 2.5
def rect(cxy, width, height, angle):
    return Rectangle(
        (cxy[0] - width/2, cxy[1] - height/2),
        width,
        height,
        angle=angle,
        rotation_point=(0,0)
    )

In [ ]:
plt.figure(figsize=(5, 5))
ax = plt.gca()
ax.set_aspect("equal")
ax.set_xlim(-6, 6)
ax.set_ylim(-6, 6)
ax.add_patch(
    Annulus((0,0), 4.9, 0.452)
)
ax.add_patch(rect((+0.4, +2.5), 0.05, 5.0, 45))
ax.add_patch(rect((-0.4, +2.5), 0.05, 5.0, 45))
ax.add_patch(rect((+0.4, -2.5), 0.05, 5.0, 45))
ax.add_patch(rect((-0.4, -2.5), 0.05, 5.0, 45))
ax.add_patch(rect((+2.5, +0.4), 5.0, 0.05, 45))
ax.add_patch(rect((+2.5, -0.4), 5.0, 0.05, 45))
ax.add_patch(rect((-2.5, +0.4), 5.0, 0.05, 45))
ax.add_patch(rect((-2.5, -0.4), 5.0, 0.05, 45))
plt.show()

In [ ]:
# Now need to think in 3D
# strut locations are still
# (+/- 0.4, +/- 2.5) and transpositions.
# height is close to 7.0 meters.  We'll work with that.
# So defining values are:
# r0 = (0.4, 2.5, 7.0)  (but rotated 45deg)
# v0 = (0.0, 1.0, 0.0)
# thickness = 0.05
# and rotations / transposes



# For the upper baffle, lower height is
# 7.3 m (at 4.817 m) and the upper height is 8.3 m (at 2.0255 m)
# So at 2.5 m, the height is 8.13 m
# So definition is
# r0 = (0.4, 2.5, 8.13)
# v0 = (0.0, 2.792, -1.0)
# width = 0.05
# height = 2.5 should work.

# Should we try these coords in ipv?

In [ ]:
import batoid
import ipyvolume as ipv

In [ ]:
telescope = batoid.Optic.fromYaml("LSST_r.yaml")

In [ ]:
def drawCircularBaffle(ax, height, inner, outer, **kwargs):
    th = np.linspace(0, 2*np.pi, 1000)
    sth, cth = np.sin(th), np.cos(th)
    ax.plot(inner*cth, inner*sth, height, **kwargs)
    ax.plot(outer*cth, outer*sth, height, **kwargs)

In [ ]:
def rotxy(r, angle):
    sth, cth = np.sin(np.deg2rad(angle)), np.cos(np.deg2rad(angle))
    x = r[0]*cth + r[1]*sth
    y = -r[0]*sth + r[1]*cth
    z = r[2]
    return np.array([x, y, z])

In [ ]:
def drawSpiderVane(ax, r0, v0, width, length, angle=0, **kwargs):  # Worry about angle later...
    r0 = np.array(r0, dtype=float)
    v0 = np.array(v0, dtype=float)
    v0 /= np.linalg.norm(v0)
    # Find direction perp to v0 and (0, 0, 1)
    perp = np.cross(v0, (0.0, 0.0, 1.0))
    perp /= np.linalg.norm(perp)
    centerline = np.array([r0 - v0*length/2, r0 + v0*length/2])
    ax.plot(*rotxy(centerline.T-(perp*width/2)[:, None], angle), **kwargs)
    ax.plot(*rotxy(centerline.T+(perp*width/2)[:, None], angle), **kwargs)
    # return centerline.T

In [ ]:
fig = ipv.figure(width=800)
telescope.draw3d(ipv, color="blue")
for height, inner, outer in [
    (8.063, 4.448, 4.9),
    (7.263, 4.423, 5.025),
    (5.377, 4.357, 5.125),
    (2.893, 4.286, 5.0),
    (0.4394, 4.18, 4.675),
]:
    drawCircularBaffle(ipv, height, inner, outer)

# Upper spiders
for s1 in [-1, 1]:
    for s2 in [-1,1]:
        for height in [8.13, 8.43]:
            drawSpiderVane(
                ipv,
                (s1*0.4, s2*2.5, height),
                (0.0, 2.792, -s2*1.0),
                0.05, 5.0,
                angle=45
            )
            drawSpiderVane(
                ipv,
                (s2*2.5, s1*0.4, height),
                (2.792, 0.0, -s2*1.0),
                0.05, 5.0,
                angle=45
            )

# Lower spiders
for s1 in [-1, 1]:
    for s2 in [-1,1]:
        for height in [6.981, 7.29]:
            drawSpiderVane(
                ipv,
                (s1*0.4, s2*2.5, height),
                (0.0, 1, 0.0),
                0.05, 5.0,
                angle=45
            )
            drawSpiderVane(
                ipv,
                (s2*2.5, s1*0.4, height),
                (1, 0.0, 0.0),
                0.05, 5.0,
                angle=45
            )

# M2 Baffle
for i in range(21):
    drawCircularBaffle(
        ipv,
        5.352 + i/20 * (6.3747 - 5.352),
        2.3885 + i/20 * (1.71 - 2.3885),
        2.3885 + i/20 * (1.71 - 2.3885)+0.03,
    )

ipv.xlim(-6, 6)
ipv.ylim(-6, 6)
ipv.zlim(-1, 11)
ipv.style.axes_off()
# ipv.style.box_off()
fig

In [ ]:
# Above 3D model looks okay.
# Now need to project from definition onto the pupil
# Take 2 points on an edge and project them.  Then connect with a line.
# Question: Do we still get rectangles out?  I think so...

def projectSpiderVane(
    r0, v0, width, length, angle, thx, thy
):
    r0 = np.array(r0, dtype=float)
    v0 = np.array(v0, dtype=float)
    v0 /= np.linalg.norm(v0)
    # Find direction perp to v0 and (0, 0, 1)
    perp = np.cross(v0, (0.0, 0.0, 1.0))
    perp /= np.linalg.norm(perp)
    centerline = np.array([r0 - v0*length/2, r0 + v0*length/2])
    edge1 = rotxy(centerline.T-(perp*width/2)[:, None], angle)
    edge2 = rotxy(centerline.T+(perp*width/2)[:, None], angle)
    # Projection vector
    v = batoid.utils.gnomonicToDirCos(np.deg2rad(thx), np.deg2rad(thy))

    t1 = -edge1[2]/v[2]
    t2 = -edge2[2]/v[2]
    proj1 = (edge1 + np.outer(v, t1))
    proj2 = (edge2 + np.outer(v, t2))
    return proj1[:2], proj2[:2]

In [ ]:
@ipywidgets.interact(
    th = ipywidgets.FloatText(step=15),
    ph = ipywidgets.FloatText(step=0.25),
    angle = ipywidgets.FloatText(step=15),
)
def f(th, ph, angle):
    thx = ph * np.cos(np.deg2rad(th))
    thy = ph * np.sin(np.deg2rad(th))
    plt.figure(figsize=(5, 5))
    ax = plt.gca()
    ax.set_aspect("equal")
    ax.set_xlim(-5, 5)
    ax.set_ylim(-5, 5)
    # M1
    ax.add_patch(
        Annulus((0,0), 4.18, 4.18-2.558, alpha=0.2, color="blue")
    )

    for s1 in [-1, 1]:
        for s2 in [-1,1]:
            # Lower spider
            for height in [6.98, 7.29]:
                for line in projectSpiderVane(
                    (s1*0.4, s2*2.5, height),
                    (0.0, 1, 0.0),
                    0.05, 5.0,
                    angle=angle,
                    thx=thx, thy=thy
                ):
                    ax.plot(*line, c='r', lw=0.5)
                for line in projectSpiderVane(
                    (s2*2.5, s1*0.4, height),
                    (1, 0.0, 0.0),
                    0.05, 5.0,
                    angle=angle,
                    thx=thx, thy=thy
                ):
                    ax.plot(*line, c='r', lw=0.5)
            # Upper spider
            for height in [8.13, 8.43]:
                for line in projectSpiderVane(
                    (s1*0.4, s2*2.5, height),
                    (0.0, 2.792, -s2*1.0),
                    0.05, 5.0,
                    angle=angle,
                    thx=thx, thy=thy
                ):
                    ax.plot(*line, c='m', lw=0.5)
                for line in projectSpiderVane(
                    (s2*2.5, s1*0.4, height),
                    (2.792, 0.0, -s2*1.0),
                    0.05, 5.0,
                    angle=angle,
                    thx=thx, thy=thy
                ):
                    ax.plot(*line, c='m', lw=0.5)

    ax.axhline(0, c='k', lw=0.5)
    ax.axvline(0, c='k', lw=0.5)
    plt.show()

In [ ]:
#3d information in the yaml.

In [ ]:
# Implementation should be to cutout area between two line segments.
# Let's start assuming parallel lines.
# Define by point, angle, width, length.
# Only consider points close to the centerline segment.  Should handle the endpoints.
# All pixels far from the segment are 1.0
# All pixels below top line get flipped.
# All pixels below bottom line are flipped again.

# What does this look like if the lines are not parallel?
# Still def